In [2]:
import os, sys
import dotenv
dotenv.load_dotenv('../.env')

import numpy as np
import torch
import pandas as pd
from scipy.stats import kendalltau, pearsonr, spearmanr
import pandas as pd
from torchvision import transforms

import wandb
from collections import defaultdict
import json
from pathlib import Path
from tqdm import tqdm
from copy import deepcopy
import warnings

sys.path.append('../')
from common import OnnxModel
from metrics.metrics import calculate_open_loop_metrics, calculate_closed_loop_metrics
from dataloading.nvidia import NvidiaElvaDataset, NvidiaDataset

WANDB_ENTITY = os.getenv('WANDB_ENTITY')
WANDB_PROJECT = os.getenv('WANDB_PROJECT')

In [14]:
def wandb_model_path_parse(model_path):
    return model_path.replace('_models/', '').replace('.onnx', '')

def add_wandb_metrics(df):
    '''Adds eval metrics to the table, such as best MAE and Vista performance.
    '''
    api = wandb.Api()
    runs = api.runs(f'{WANDB_ENTITY}/{WANDB_PROJECT}')

    for run in runs:

        # training run
        any_drives_with_this_model = df.loc[df['training_run'] == run.id].size > 0
        if any_drives_with_this_model:
            best_mae = 2**32
            for i, row in run.history(keys=['mae']).iterrows():
                if row['mae'] < best_mae: best_mae = row['mae']

            df.loc[df['training_run'] == run.id, 'val_mae'] = best_mae
            df.loc[df['training_run'] == run.id, 'model_type'] = run.config['model_type']

        # evaluation run
        if run.state == 'finished' and 'offline-elva-evaluation' in run.tags:
            model_name = wandb_model_path_parse(run.config['model_path'])
            df.loc[df['model_name'] == model_name, 'elva_mae'] = run.summary['mae']
            df.loc[df['model_name'] == model_name, 'elva_whiteness'] = run.summary['whiteness']
            df.loc[df['model_name'] == model_name, 'elva_expert_whiteness'] = run.summary['expert_whiteness']

    df['real_interventions'] = df['real_interventions'].convert_dtypes()
    return df

In [15]:
# ONLINE METRICS

track_direction_change_location = np.array([-9683.68050786, -1542.68155186])
root_path = Path("/data/Bolt/end-to-end/drives-ebm-paper/")
expert_ds = NvidiaDataset([root_path / '2021-10-26-10-49-06_e2e_rec_ss20_elva_eval_chunk'])
expert_back_ds = NvidiaDataset([root_path / '2021-10-26-11-08-59_e2e_rec_ss20_elva_back_eval_chunk'])

def are_locations_close(loc_a, loc_b, threshold=50):
    return np.linalg.norm(loc_a - loc_b) < threshold

def get_closest_frame_by_loc(df, target_loc):
    locations = df[['position_x', 'position_y']].to_numpy().astype(np.float32)
    df['distance_to_target'] = np.linalg.norm(locations - target_loc)
    return df.loc[df['distance_to_target'].idxmin()]

def get_closest_row_idx_by_timestamp(df, dt):
    df['timestamp'] = pd.to_datetime(df['index'])
    return (abs(df['timestamp'] - dt)).idxmin()

def get_longest_intervention_periods(df):
    df['autonomous_next'] = df['autonomous'].shift(-1)
    starts_ends_df = df[(df['autonomous'] & (df['autonomous_next'] == False)) | ((df['autonomous'] == False) & df['autonomous_next'])]
    starts_ends = [row['row_id'] for i, row in starts_ends_df.iterrows()]
    starts = np.array(starts_ends)[::2]
    ends = np.array(starts_ends)[1::2]
    longest_idxs = np.argsort(ends - starts)
    return (starts[longest_idxs], ends[longest_idxs])

def split_back_forth_drive_into_two(dataset):

    frames_df = dataset.frames
    vehicle_cmd_df = dataset.vehicle_cmd_frames
    # find the longest intervention period
    found_direction_change = False
    for forward_end, forward_start in zip(*get_longest_intervention_periods(frames_df)):
        if are_locations_close(frames_df[frames_df['row_id'] == forward_end][['position_x', 'position_y']].to_numpy(), track_direction_change_location) or \
            are_locations_close(frames_df[frames_df['row_id'] == forward_start][['position_x', 'position_y']].to_numpy(), track_direction_change_location):
            found_direction_change = True
            break

    if not found_direction_change:
        print('Couldn\'t find the longest intervention in the track direction change location')
        return None

    # split the drive into two
    df1 = frames_df[frames_df['row_id'] <= forward_end]
    df2 = frames_df[frames_df['row_id'] > forward_start]

    forward_end_ts = pd.to_datetime(df1.iloc[-1]['index'])
    backward_start_ts = pd.to_datetime(df2.iloc[0]['index'])

    forward_end_idx = get_closest_row_idx_by_timestamp(vehicle_cmd_df, forward_end_ts)
    backward_end_idx = get_closest_row_idx_by_timestamp(vehicle_cmd_df, backward_start_ts)

    df1_vehicle_cmd = vehicle_cmd_df.iloc[:forward_end_idx]
    df2_vehicle_cmd = vehicle_cmd_df.iloc[backward_end_idx:]

    # save the pandas dataframes back into NvidiaDataset objects
    dataset_forward = deepcopy(dataset)
    dataset_backward = deepcopy(dataset)
    dataset_forward.frames = df1
    dataset_forward.vehicle_cmd_frames = df1_vehicle_cmd
    dataset_backward.frames = df2
    dataset_backward.vehicle_cmd_frames = df2_vehicle_cmd

    return dataset_forward, dataset_backward

def add_online_metrics(df):
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        for i, row in tqdm(df.iterrows()):
            drives = row['drive']
            ds_forward = None
            ds_backward = None
            forward_metrics = {}
            backward_metrics = {}
            if len(drives) == 1:
                ds_combined = NvidiaDataset([root_path / drives[0]])
                if drives[0] not in ['2022-09-09-11-47-04', '2022-09-09-10-51-33-mdn-1-s1']: # unfinished single direction drive
                    ds_forward, ds_backward = split_back_forth_drive_into_two(ds_combined)
                else:
                    ds_forward = ds_combined
            elif len(drives) == 2:
                ds_forward = NvidiaDataset([root_path / drives[0]])
                ds_backward = NvidiaDataset([root_path / drives[1]])
            elif len(drives) == 3:
                assert '2022-08-31-15-18-55_elva_classifier_512_forward_continued' in drives[1]
                ds_forward = NvidiaDataset([root_path / drives[0], root_path / drives[1]])
                ds_backward = NvidiaDataset([root_path / drives[2]])

            if ds_forward:
                forward_metrics = calculate_closed_loop_metrics(ds_forward.frames, expert_ds.frames, ds_forward.vehicle_cmd_frames)
            if ds_backward:
                backward_metrics = calculate_closed_loop_metrics(ds_backward.frames, expert_back_ds.frames, ds_backward.vehicle_cmd_frames)

            try:
                del forward_metrics['interventions']
                del backward_metrics['interventions']
            except:
                pass

            metrics = {**forward_metrics}
            additive_metrics = ['distance']
            for k, v in backward_metrics.items():
                metrics[k] = metrics[k] + v
                if k not in additive_metrics:
                    metrics[k] = metrics[k] / 2

            metrics['distance_per_intervention'] = metrics['distance'] / row['real_interventions']
            df.loc[i, metrics.keys()] = metrics.values()
            
    return df

[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/2021-10-26-10-49-06_e2e_rec_ss20_elva_eval_chunk: length=13858, filtered=2
[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/2021-10-26-11-08-59_e2e_rec_ss20_elva_back_eval_chunk: length=14249, filtered=1


In [54]:
# columns = ['model_name', 'model_class', 'total_MAE', 'same_track_MAE', 'vista_interventions', 'real_interventions']
columns = ['training_run', 'model_name', 'real_interventions', 'drive']

# build a dataframe with the results
rows = [
    ['3ftnqxcb', 'ebm-512-s1', 9, ('e2e-ebm-512-s1-day1-forward_2022-09-20-11-00-41', 'e2e-ebm-512-s1-day1-backward_2022-09-20-11-11-14')],
    ['3ftnqxcb', 'ebm-512-s1', 4, ('ebm-paper-ebm-512-s1-forward_2022-09-21-10-51-39', 'ebm-paper-ebm-512-s1-backward_2022-09-21-11-02-14')],
    ['3ftnqxcb', 'ebm-512-s1', 1, ('ebm-paper-classifier-512-forward_2022-09-22-11-47-18', 'ebm-paper-classifier-512-backward_2022-09-22-11-58-03')],
    ['3ftnqxcb', 'ebm-512-s1', 2, ('ebm-paper-ebm-512-s1-forward_2022-09-23-11-18-29', 'ebm-paper-ebm-512-s1-backwards_2022-09-23-11-07-13')],

    ['3jk7cnqa', 'ebm-normal-1-s1', 5, ('e2e-ebm-normal-1-s1-day1-forward_2022-09-20-13-37-56', 'e2e-ebm-normal-1-s1-day1-backward_2022-09-20-13-48-42')],
    ['3jk7cnqa', 'ebm-normal-1-s1', 20, ('ebm-paper-ebm-normal-1-s1-forward_2022-09-21-11-59-17', 'ebm-paper-ebm-normal-1-s1-backward_2022-09-21-12-10-33')], # RUINED BY RAIN
    ['3jk7cnqa', 'ebm-normal-1-s1', 2, ('ebm-paper-ebm-normal-1-s1-forward_2022-09-22-12-15-22', 'ebm-paper-ebm-normal-1-s1-backward_2022-09-22-12-25-57')],
    ['3jk7cnqa', 'ebm-normal-1-s1', 3, ('ebm-paper-ebm-normal-1-s1-forward_2022-09-23-12-28-45', 'ebm-paper-ebm-normal-1-s1-backwards_2022-09-23-12-17-08')],

    ['2jvl4yhn', 'ebm-spatial-0-s2', 5, ('e2e-ebm-spatial-0-s2-day1-forward_2022-09-20-12-44-32', 'e2e-ebm-spatial-0-s2-day1-backward_2022-09-20-12-55-04')],
    ['2jvl4yhn', 'ebm-spatial-0-s2', 5, ('ebm-paper-ebm-spatial-0-s2-forward_2022-09-21-11-13-54', 'ebm-paper-ebm-spatial-0-s2-backward_2022-09-21-11-24-29')],
    ['2jvl4yhn', 'ebm-spatial-0-s2', 8, ('ebm-paper-ebm-spatial-0-s2-forward_2022-09-22-11-25-33', 'ebm-paper-ebm-spatial-0-s2-backward_2022-09-22-11-36-02')],
    ['2jvl4yhn', 'ebm-spatial-0-s2', 4, ('ebm-paper-ebm-spatial-0-s2-forward_2022-09-23-11-43-09', 'ebm-paper-ebm-spatial-0-s2-backwards_2022-09-23-11-31-39')],

    ['bxd5wtqk', 'mae-s2', 2, ('e2e-mae-s2-day1-forward_2022-09-20-11-25-14', 'e2e-mae-s2-day1-backward_2022-09-20-11-35-49')],
    ['bxd5wtqk', 'mae-s2', 2, ('ebm-paper-mae-s2-forward_2022-09-21-12-22-57', 'ebm-paper-mae-s2-backward_2022-09-21-12-34-47')],
    ['bxd5wtqk', 'mae-s2', 5, ('ebm-paper-mae-s2-forward_2022-09-22-10-02-03', 'ebm-paper-mae-s2-backward_2022-09-22-10-26-02')],
    ['bxd5wtqk', 'mae-s2', 1, ('ebm-paper-mae-s2-forward_2022-09-23-10-31-24', 'ebm-paper-mae-s2-backward_2022-09-23-10-19-55')],

    ['3g3wwx73', 'classifier-512', 7, ('e2e-classifier-512-day1-forward_2022-09-20-12-20-47', 'e2e-classifier-512-day1-backward_2022-09-20-12-31-35')],
    ['3g3wwx73', 'classifier-512', 1, ('ebm-paper-classifier-512-forward_2022-09-21-11-36-50', 'ebm-paper-classifier-512-backward_2022-09-21-11-47-38')],
    ['3g3wwx73', 'classifier-512', 7, ('ebm-paper-classifier-512-forward_2022-09-22-11-47-18', 'ebm-paper-classifier-512-backward_2022-09-22-11-58-03')],
    ['3g3wwx73', 'classifier-512', 1, ('ebm-paper-classifier-512-forward_2022-09-23-12-05-48', 'ebm-paper-classifier-512-backwards_2022-09-23-11-54-32')],

    ['1hbbr6dm', 'mdn-5-s1', 10, ('e2e-mdn-5-s1-day1-forward_2022-09-20-14-00-44', 'e2e-mdn-5-s1-day1-backward_2022-09-20-14-13-08')], # RUINED BY RAIN
    ['1hbbr6dm', 'mdn-5-s1', 1,  ('ebm-paper-mdn-5-s1-forward_2022-09-21-10-29-09', 'ebm-paper-mdn-5-s1-backward_2022-09-21-10-39-43')],
    ['1hbbr6dm', 'mdn-5-s1', 5, ('ebm-paper-mdn-5-s1-forward_2022-09-22-10-38-10', 'ebm-paper-mdn-5-s1-backward_2022-09-22-10-49-36')],
    ['1hbbr6dm', 'mdn-5-s1', 5, ('ebm-paper-mdn-5-s1-forward_2022-09-23-10-55-27', 'ebm-paper-mdn-5-s1-backwards_2022-09-23-10-43-34')],
]

df = pd.DataFrame(rows, columns=columns)
df.loc[df['drive'] == ('e2e-ebm-512-s1-day1-forward_2022-09-20-11-00-41', 'e2e-ebm-512-s1-day1-backward_2022-09-20-11-11-14'), 'comment'] = 'strict Tambet\'s first drive of the day'
df.loc[df['drive'] == ('ebm-paper-ebm-normal-1-s1-forward_2022-09-21-11-59-17', 'ebm-paper-ebm-normal-1-s1-backward_2022-09-21-12-10-33'), 'comment'] = 'ruined by rain'
df.loc[df['drive'] == ('e2e-mdn-5-s1-day1-forward_2022-09-20-14-00-44', 'e2e-mdn-5-s1-day1-backward_2022-09-20-14-13-08'), 'comment'] = 'ruined by rain'
df.loc[df['drive'] == ('ebm-paper-mae-s2-forward_2022-09-22-10-02-03', 'ebm-paper-mae-s2-backward_2022-09-22-10-26-02'), 'comment'] = 'sunny'
df.loc[df['drive'] == ('ebm-paper-classifier-512-forward_2022-09-22-11-47-18', 'ebm-paper-classifier-512-backward_2022-09-22-11-58-03'), 'comment'] = 'sunny'
df.loc[df['drive'] == ('e2e-classifier-512-day1-forward_2022-09-20-12-20-47', 'e2e-classifier-512-day1-backward_2022-09-20-12-31-35'), 'comment'] = 'sunny'

# show df without column "training_run"
df.drop(columns=['training_run', 'drive'])

,model_name,real_interventions,comment
0,ebm-512-s1,9,strict Tambet's first drive of the day
1,ebm-512-s1,4,NaN
2,ebm-512-s1,1,sunny
3,ebm-512-s1,2,NaN
4,ebm-normal-1-s1,5,NaN
5,ebm-normal-1-s1,20,ruined by rain
6,ebm-normal-1-s1,2,NaN
7,ebm-normal-1-s1,3,NaN
8,ebm-spatial-0-s2,5,NaN
9,ebm-spatial-0-s2,5,NaN


In [55]:
columns = ['training_run', 'model_name', 'real_interventions', 'drive']

# build a dataframe with the results
rows = [
    # ['3ftnqxcb', 'ebm-512-s1', 9, ('e2e-ebm-512-s1-day1-forward_2022-09-20-11-00-41', 'e2e-ebm-512-s1-day1-backward_2022-09-20-11-11-14')],
    ['3ftnqxcb', 'ebm-512-s1', 4, ('ebm-paper-ebm-512-s1-forward_2022-09-21-10-51-39', 'ebm-paper-ebm-512-s1-backward_2022-09-21-11-02-14')],
    ['3ftnqxcb', 'ebm-512-s1', 1, ('ebm-paper-classifier-512-forward_2022-09-22-11-47-18', 'ebm-paper-classifier-512-backward_2022-09-22-11-58-03')],
    ['3ftnqxcb', 'ebm-512-s1', 2, ('ebm-paper-ebm-512-s1-forward_2022-09-23-11-18-29', 'ebm-paper-ebm-512-s1-backwards_2022-09-23-11-07-13')],

    ['3jk7cnqa', 'ebm-normal-1-s1', 5, ('e2e-ebm-normal-1-s1-day1-forward_2022-09-20-13-37-56', 'e2e-ebm-normal-1-s1-day1-backward_2022-09-20-13-48-42')],
    # ['3jk7cnqa', 'ebm-normal-1-s1', 20, ('ebm-paper-ebm-normal-1-s1-forward_2022-09-21-11-59-17', 'ebm-paper-ebm-normal-1-s1-backward_2022-09-21-12-10-33')], # RUINED BY RAIN
    ['3jk7cnqa', 'ebm-normal-1-s1', 2, ('ebm-paper-ebm-normal-1-s1-forward_2022-09-22-12-15-22', 'ebm-paper-ebm-normal-1-s1-backward_2022-09-22-12-25-57')],
    ['3jk7cnqa', 'ebm-normal-1-s1', 3, ('ebm-paper-ebm-normal-1-s1-forward_2022-09-23-12-28-45', 'ebm-paper-ebm-normal-1-s1-backwards_2022-09-23-12-17-08')],

    ['2jvl4yhn', 'ebm-spatial-0-s2', 5, ('e2e-ebm-spatial-0-s2-day1-forward_2022-09-20-12-44-32', 'e2e-ebm-spatial-0-s2-day1-backward_2022-09-20-12-55-04')],
    ['2jvl4yhn', 'ebm-spatial-0-s2', 5, ('ebm-paper-ebm-spatial-0-s2-forward_2022-09-21-11-13-54', 'ebm-paper-ebm-spatial-0-s2-backward_2022-09-21-11-24-29')],
    # ['2jvl4yhn', 'ebm-spatial-0-s2', 8, ('ebm-paper-ebm-spatial-0-s2-forward_2022-09-22-11-25-33', 'ebm-paper-ebm-spatial-0-s2-backward_2022-09-22-11-36-02')],
    ['2jvl4yhn', 'ebm-spatial-0-s2', 4, ('ebm-paper-ebm-spatial-0-s2-forward_2022-09-23-11-43-09', 'ebm-paper-ebm-spatial-0-s2-backwards_2022-09-23-11-31-39')],

    ['bxd5wtqk', 'mae-s2', 2, ('e2e-mae-s2-day1-forward_2022-09-20-11-25-14', 'e2e-mae-s2-day1-backward_2022-09-20-11-35-49')],
    ['bxd5wtqk', 'mae-s2', 2, ('ebm-paper-mae-s2-forward_2022-09-21-12-22-57', 'ebm-paper-mae-s2-backward_2022-09-21-12-34-47')],
    # ['bxd5wtqk', 'mae-s2', 5, ('ebm-paper-mae-s2-forward_2022-09-22-10-02-03', 'ebm-paper-mae-s2-backward_2022-09-22-10-26-02')],
    ['bxd5wtqk', 'mae-s2', 1, ('ebm-paper-mae-s2-forward_2022-09-23-10-31-24', 'ebm-paper-mae-s2-backward_2022-09-23-10-19-55')],

    ['3g3wwx73', 'classifier-512', 7, ('e2e-classifier-512-day1-forward_2022-09-20-12-20-47', 'e2e-classifier-512-day1-backward_2022-09-20-12-31-35')],
    ['3g3wwx73', 'classifier-512', 1, ('ebm-paper-classifier-512-forward_2022-09-21-11-36-50', 'ebm-paper-classifier-512-backward_2022-09-21-11-47-38')],
    ['3g3wwx73', 'classifier-512', 7, ('ebm-paper-classifier-512-forward_2022-09-22-11-47-18', 'ebm-paper-classifier-512-backward_2022-09-22-11-58-03')],
    ['3g3wwx73', 'classifier-512', 1, ('ebm-paper-classifier-512-forward_2022-09-23-12-05-48', 'ebm-paper-classifier-512-backwards_2022-09-23-11-54-32')],

    # ['1hbbr6dm', 'mdn-5-s1', 10, ('e2e-mdn-5-s1-day1-forward_2022-09-20-14-00-44', 'e2e-mdn-5-s1-day1-backward_2022-09-20-14-13-08')], # RUINED BY RAIN
    ['1hbbr6dm', 'mdn-5-s1', 1,  ('ebm-paper-mdn-5-s1-forward_2022-09-21-10-29-09', 'ebm-paper-mdn-5-s1-backward_2022-09-21-10-39-43')],
    ['1hbbr6dm', 'mdn-5-s1', 5, ('ebm-paper-mdn-5-s1-forward_2022-09-22-10-38-10', 'ebm-paper-mdn-5-s1-backward_2022-09-22-10-49-36')],
    ['1hbbr6dm', 'mdn-5-s1', 5, ('ebm-paper-mdn-5-s1-forward_2022-09-23-10-55-27', 'ebm-paper-mdn-5-s1-backwards_2022-09-23-10-43-34')],
]

df = pd.DataFrame(rows, columns=columns)
df

,training_run,model_name,real_interventions,drive
0,3ftnqxcb,ebm-512-s1,4,(ebm-paper-ebm-512-s1-forward_2022-09-21-10-51...
1,3ftnqxcb,ebm-512-s1,1,(ebm-paper-classifier-512-forward_2022-09-22-1...
2,3ftnqxcb,ebm-512-s1,2,(ebm-paper-ebm-512-s1-forward_2022-09-23-11-18...
3,3jk7cnqa,ebm-normal-1-s1,5,(e2e-ebm-normal-1-s1-day1-forward_2022-09-20-1...
4,3jk7cnqa,ebm-normal-1-s1,2,(ebm-paper-ebm-normal-1-s1-forward_2022-09-22-...
5,3jk7cnqa,ebm-normal-1-s1,3,(ebm-paper-ebm-normal-1-s1-forward_2022-09-23-...
6,2jvl4yhn,ebm-spatial-0-s2,5,(e2e-ebm-spatial-0-s2-day1-forward_2022-09-20-...
7,2jvl4yhn,ebm-spatial-0-s2,5,(ebm-paper-ebm-spatial-0-s2-forward_2022-09-21...
8,2jvl4yhn,ebm-spatial-0-s2,4,(ebm-paper-ebm-spatial-0-s2-forward_2022-09-23...
9,bxd5wtqk,mae-s2,2,"(e2e-mae-s2-day1-forward_2022-09-20-11-25-14, ..."


In [56]:
df.groupby('model_name').mean()

,real_interventions
model_name,
classifier-512,4.0000
ebm-512-s1,2.3333
ebm-normal-1-s1,3.3333
ebm-spatial-0-s2,4.6667
mae-s2,1.6667
mdn-5-s1,3.6667


In [58]:
df.to_csv('ebm-experiments-final.csv', index=False)

In [59]:
df = add_wandb_metrics(df)
df = add_online_metrics(df)
df.to_csv('ebm-experiments-final-results.csv', index=False)
df

0it [00:00, ?it/s]

[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-ebm-512-s1-forward_2022-09-21-10-51-39: length=17873, filtered=0
[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-ebm-512-s1-backward_2022-09-21-11-02-14: length=18309, filtered=0


1it [00:25, 25.17s/it]

[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-classifier-512-forward_2022-09-22-11-47-18: length=18334, filtered=0
[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-classifier-512-backward_2022-09-22-11-58-03: length=18259, filtered=0


2it [00:50, 25.10s/it]

...

[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-mdn-5-s1-forward_2022-09-22-10-38-10: length=17961, filtered=0
[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-mdn-5-s1-backward_2022-09-22-10-49-36: length=18523, filtered=0


18it [07:39, 25.56s/it]

[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-mdn-5-s1-forward_2022-09-23-10-55-27: length=18415, filtered=0
[NvidiaDataset] Using default transform: Compose(
)
/data/Bolt/end-to-end/drives-ebm-paper/ebm-paper-mdn-5-s1-backwards_2022-09-23-10-43-34: length=18635, filtered=0


19it [08:04, 25.51s/it]


,training_run,model_name,real_interventions,drive,elva_mae,elva_whiteness,elva_expert_whiteness,val_mae,model_type,traj_mae,traj_rmse,traj_max,traj_failure_rate,distance,distance_per_intervention,whiteness,cmd_whiteness,expert_whiteness
0,3ftnqxcb,ebm-512-s1,4,(ebm-paper-ebm-512-s1-forward_2022-09-21-10-51...,7.4874,191.0606,17.6794,8.6071,pilotnet-ebm,0.5021,0.6178,1.7164,10.5927,8479.2291,2119.8073,35.2479,176.9281,17.6551
1,3ftnqxcb,ebm-512-s1,1,(ebm-paper-classifier-512-forward_2022-09-22-1...,7.4874,191.0606,17.6794,8.6071,pilotnet-ebm,0.4739,0.5898,1.6061,9.6901,8453.5230,8453.5230,62.1674,287.1409,17.6551
2,3ftnqxcb,ebm-512-s1,2,(ebm-paper-ebm-512-s1-forward_2022-09-23-11-18...,7.4874,191.0606,17.6794,8.6071,pilotnet-ebm,0.4829,0.5927,1.7184,9.0726,8519.6575,4259.8287,28.5703,223.5885,17.6551
3,3jk7cnqa,ebm-normal-1-s1,5,(e2e-ebm-normal-1-s1-day1-forward_2022-09-20-1...,6.9747,152.1919,17.6794,8.4375,pilotnet-ebm,0.4124,0.5250,1.7214,5.8364,8473.0965,1694.6193,49.9224,119.3907,17.6551
4,3jk7cnqa,ebm-normal-1-s1,2,(ebm-paper-ebm-normal-1-s1-forward_2022-09-22-...,6.9747,152.1919,17.6794,8.4375,pilotnet-ebm,0.4080,0.5127,1.7083,5.5971,8473.6965,4236.8483,38.9566,137.2209,17.6551
5,3jk7cnqa,ebm-normal-1-s1,3,(ebm-paper-ebm-normal-1-s1-forward_2022-09-23-...,6.9747,152.1919,17.6794,8.4375,pilotnet-ebm,0.4478,0.5528,1.5968,7.9200,8520.9768,2840.3256,34.2063,77.2751,17.6551
6,2jvl4yhn,ebm-spatial-0-s2,5,(e2e-ebm-spatial-0-s2-day1-forward_2022-09-20-...,6.9099,125.8168,17.6794,8.7380,pilotnet-ebm,0.4364,0.5483,1.6401,6.9096,8444.2192,1688.8438,27.7969,56.3284,17.6551
7,2jvl4yhn,ebm-spatial-0-s2,5,(ebm-paper-ebm-spatial-0-s2-forward_2022-09-21...,6.9099,125.8168,17.6794,8.7380,pilotnet-ebm,0.4426,0.5595,1.5098,8.5461,8442.2459,1688.4492,46.8295,57.1483,17.6551
8,2jvl4yhn,ebm-spatial-0-s2,4,(ebm-paper-ebm-spatial-0-s2-forward_2022-09-23...,6.9099,125.8168,17.6794,8.7380,pilotnet-ebm,0.4644,0.5831,1.6892,8.9059,8466.3047,2116.5762,33.7233,56.8619,17.6551
9,bxd5wtqk,mae-s2,2,"(e2e-mae-s2-day1-forward_2022-09-20-11-25-14, ...",7.1969,54.8467,17.6794,7.8280,pilotnet,0.4010,0.5044,1.5977,5.8486,8516.5109,4258.2554,26.6905,37.8366,17.6551


In [65]:

grouped_df = df.groupby(['model_type', 'model_name'])

mean_vals = grouped_df.mean().sort_values(by=['real_interventions'])
median_vals = grouped_df.median().sort_values(by=['real_interventions'])
sorted_df = df.sort_values(by=['real_interventions'])
min_vals = sorted_df.groupby(['model_type', 'model_name']).first().sort_values(by=['real_interventions'])

mean_vals['distance_per_intervention'] = mean_vals['distance'] / mean_vals['real_interventions']
median_vals['distance_per_intervention'] = median_vals['distance'] / median_vals['real_interventions']
min_vals['distance_per_intervention'] = min_vals['distance'] / min_vals['real_interventions']

print('Mean:')
display(mean_vals)
print('Median:')
display(median_vals)
print('Best of each model:')
display(min_vals)

Mean:


real_interventions  elva_mae  \
model_type          model_name                                       
pilotnet            mae-s2                        1.6667    7.1969   
pilotnet-ebm        ebm-512-s1                    2.3333    7.4874   
                    ebm-normal-1-s1               3.3333    6.9747   
pilotnet-mdn        mdn-5-s1                      3.6667    6.7350   
pilotnet-classifier classifier-512                4.0000    8.1178   
pilotnet-ebm        ebm-spatial-0-s2              4.6667    6.9099   

                                      elva_whiteness  elva_expert_whiteness  \
model_type          model_name                                                
pilotnet            mae-s2                   54.8467                17.6794   
pilotnet-ebm        ebm-512-s1              191.0606                17.6794   
                    ebm-normal-1-s1         152.1919                17.6794   
pilotnet-mdn        mdn-5-s1                 74.4168                17.6794   
pilotnet-classifier classifier-512          208.4579                17.6794   
pilotnet-ebm        ebm-spatial-0-s2        125.8168                17.6794   

                                      val_mae  traj_mae  traj_rmse  traj_max  \
model_type          model_name                                                 
pilotnet            mae-s2             7.8280    0.4287     0.5316    1.6191   
pilotnet-ebm        ebm-512-s1         8.6071    0.4863     0.6001    1.6803   
                    ebm-normal-1-s1    8.4375    0.4227     0.5302    1.6755   
pilotnet-mdn        mdn-5-s1           9.0706    0.4334     0.5423    1.6938   
pilotnet-classifier classifier-512     9.9258    0.4314     0.5398    1.6903   
pilotnet-ebm        ebm-spatial-0-s2   8.7380    0.4478     0.5636    1.6130   

                                      traj_failure_rate  distance  \
model_type          model_name                                      
pilotnet            mae-s2                       5.8036 8547.1309   
pilotnet-ebm        ebm-512-s1                   9.7851 8484.1365   
                    ebm-normal-1-s1              6.4511 8489.2566   
pilotnet-mdn        mdn-5-s1                     6.5511 8456.1635   
pilotnet-classifier classifier-512               6.5394 8500.4469   
pilotnet-ebm        ebm-spatial-0-s2             8.1205 8450.9232   

                                      distance_per_intervention  whiteness  \
model_type          model_name                                               
pilotnet            mae-s2                            5128.2786    27.5376   
pilotnet-ebm        ebm-512-s1                        3636.0585    41.9952   
                    ebm-normal-1-s1                   2546.7770    41.0284   
pilotnet-mdn        mdn-5-s1                          2306.2264    25.5995   
pilotnet-classifier classifier-512                    2125.1117    48.5076   
pilotnet-ebm        ebm-spatial-0-s2                  1810.9121    36.1166   

                                      cmd_whiteness  expert_whiteness  
model_type          model_name                                         
pilotnet            mae-s2                  48.7610           17.6551  
pilotnet-ebm        ebm-512-s1             229.2192           17.6551  
                    ebm-normal-1-s1        111.2956           17.6551  
pilotnet-mdn        mdn-5-s1                35.4902           17.6551  
pilotnet-classifier classifier-512         193.9664           17.6551  
pilotnet-ebm        ebm-spatial-0-s2        56.7795           17.6551

Median:


real_interventions  elva_mae  \
model_type          model_name                                       
pilotnet            mae-s2                        2.0000    7.1969   
pilotnet-ebm        ebm-512-s1                    2.0000    7.4874   
                    ebm-normal-1-s1               3.0000    6.9747   
pilotnet-classifier classifier-512                4.0000    8.1178   
pilotnet-ebm        ebm-spatial-0-s2              5.0000    6.9099   
pilotnet-mdn        mdn-5-s1                      5.0000    6.7350   

                                      elva_whiteness  elva_expert_whiteness  \
model_type          model_name                                                
pilotnet            mae-s2                   54.8467                17.6794   
pilotnet-ebm        ebm-512-s1              191.0606                17.6794   
                    ebm-normal-1-s1         152.1919                17.6794   
pilotnet-classifier classifier-512          208.4579                17.6794   
pilotnet-ebm        ebm-spatial-0-s2        125.8168                17.6794   
pilotnet-mdn        mdn-5-s1                 74.4168                17.6794   

                                      val_mae  traj_mae  traj_rmse  traj_max  \
model_type          model_name                                                 
pilotnet            mae-s2             7.8280    0.4409     0.5403    1.5977   
pilotnet-ebm        ebm-512-s1         8.6071    0.4829     0.5927    1.7164   
                    ebm-normal-1-s1    8.4375    0.4124     0.5250    1.7083   
pilotnet-classifier classifier-512     9.9258    0.4289     0.5355    1.7168   
pilotnet-ebm        ebm-spatial-0-s2   8.7380    0.4426     0.5595    1.6401   
pilotnet-mdn        mdn-5-s1           9.0706    0.4272     0.5317    1.7188   

                                      traj_failure_rate  distance  \
model_type          model_name                                      
pilotnet            mae-s2                       5.8486 8558.5168   
pilotnet-ebm        ebm-512-s1                   9.6901 8479.2291   
                    ebm-normal-1-s1              5.8364 8473.6965   
pilotnet-classifier classifier-512               5.7424 8493.7997   
pilotnet-ebm        ebm-spatial-0-s2             8.5461 8444.2192   
pilotnet-mdn        mdn-5-s1                     5.7820 8433.0665   

                                      distance_per_intervention  whiteness  \
model_type          model_name                                               
pilotnet            mae-s2                            4279.2584    26.6905   
pilotnet-ebm        ebm-512-s1                        4239.6146    35.2479   
                    ebm-normal-1-s1                   2824.5655    38.9566   
pilotnet-classifier classifier-512                    2123.4499    48.8780   
pilotnet-ebm        ebm-spatial-0-s2                  1688.8438    33.7233   
pilotnet-mdn        mdn-5-s1                          1686.6133    25.3156   

                                      cmd_whiteness  expert_whiteness  
model_type          model_name                                         
pilotnet            mae-s2                  37.8366           17.6551  
pilotnet-ebm        ebm-512-s1             223.5885           17.6551  
                    ebm-normal-1-s1        119.3907           17.6551  
pilotnet-classifier classifier-512         172.3336           17.6551  
pilotnet-ebm        ebm-spatial-0-s2        56.8619           17.6551  
pilotnet-mdn        mdn-5-s1                35.4609           17.6551

Best of each model:


training_run  real_interventions  \
model_type          model_name                                          
pilotnet            mae-s2               bxd5wtqk                   1   
pilotnet-classifier classifier-512       3g3wwx73                   1   
pilotnet-ebm        ebm-512-s1           3ftnqxcb                   1   
pilotnet-mdn        mdn-5-s1             1hbbr6dm                   1   
pilotnet-ebm        ebm-normal-1-s1      3jk7cnqa                   2   
                    ebm-spatial-0-s2     2jvl4yhn                   4   

                                                                                  drive  \
model_type          model_name                                                            
pilotnet            mae-s2            (ebm-paper-mae-s2-forward_2022-09-23-10-31-24,...   
pilotnet-classifier classifier-512    (ebm-paper-classifier-512-forward_2022-09-23-1...   
pilotnet-ebm        ebm-512-s1        (ebm-paper-classifier-512-forward_2022-09-22-1...   
pilotnet-mdn        mdn-5-s1          (ebm-paper-mdn-5-s1-forward_2022-09-21-10-29-0...   
pilotnet-ebm        ebm-normal-1-s1   (ebm-paper-ebm-normal-1-s1-forward_2022-09-22-...   
                    ebm-spatial-0-s2  (ebm-paper-ebm-spatial-0-s2-forward_2022-09-23...   

                                      elva_mae  elva_whiteness  \
model_type          model_name                                   
pilotnet            mae-s2              7.1969         54.8467   
pilotnet-classifier classifier-512      8.1178        208.4579   
pilotnet-ebm        ebm-512-s1          7.4874        191.0606   
pilotnet-mdn        mdn-5-s1            6.7350         74.4168   
pilotnet-ebm        ebm-normal-1-s1     6.9747        152.1919   
                    ebm-spatial-0-s2    6.9099        125.8168   

                                      elva_expert_whiteness  val_mae  \
model_type          model_name                                         
pilotnet            mae-s2                          17.6794   7.8280   
pilotnet-classifier classifier-512                  17.6794   9.9258   
pilotnet-ebm        ebm-512-s1                      17.6794   8.6071   
pilotnet-mdn        mdn-5-s1                        17.6794   9.0706   
pilotnet-ebm        ebm-normal-1-s1                 17.6794   8.4375   
                    ebm-spatial-0-s2                17.6794   8.7380   

                                      traj_mae  traj_rmse  traj_max  \
model_type          model_name                                        
pilotnet            mae-s2              0.4409     0.5403    1.5592   
pilotnet-classifier classifier-512      0.4244     0.5274    1.7158   
pilotnet-ebm        ebm-512-s1          0.4739     0.5898    1.6061   
pilotnet-mdn        mdn-5-s1            0.4272     0.5317    1.7188   
pilotnet-ebm        ebm-normal-1-s1     0.4080     0.5127    1.7083   
                    ebm-spatial-0-s2    0.4644     0.5831    1.6892   

                                      traj_failure_rate  distance  \
model_type          model_name                                      
pilotnet            mae-s2                       5.3619 8566.3651   
pilotnet-classifier classifier-512               5.5481 8560.6653   
pilotnet-ebm        ebm-512-s1                   9.6901 8453.5230   
pilotnet-mdn        mdn-5-s1                     4.9389 8517.8758   
pilotnet-ebm        ebm-normal-1-s1              5.5971 8473.6965   
                    ebm-spatial-0-s2             8.9059 8466.3047   

                                      distance_per_intervention  whiteness  \
model_type          model_name                                               
pilotnet            mae-s2                            8566.3651    26.2771   
pilotnet-classifier classifier-512                    8560.6653    34.1070   
pilotnet-ebm        ebm-512-s1                        8453.5230    62.1674   
pilotnet-mdn        mdn-5-s1                          8517.8758    25.3156   
pilotnet-ebm        ebm-nor

### Correlation study

In [66]:
def calculate_pvalues(df, method='pearson'):
    corr_fn = pearsonr if method == 'pearson' else spearmanr
    df = df.dropna()._get_numeric_data()
    dfcols = pd.DataFrame(columns=df.columns)
    pvalues = dfcols.transpose().join(dfcols, how='outer')
    for r in df.columns:
        for c in df.columns:
            pvalues[r][c] = round(corr_fn(df[r], df[c])[1], 4)
    return pvalues

def kendall_pval(x,y):
    return kendalltau(x,y)[1]

def pearsonr_pval(x,y):
    return pearsonr(x,y)[1]

def spearmanr_pval(x,y):
    return spearmanr(x,y)[1]

#### On-policy

In [67]:
target_cols = ['real_interventions']
candidate_cols = ['traj_mae', 'whiteness', 'cmd_whiteness']

print('pearson')
display(df.dropna().groupby('model_name').mean().corr('pearson').loc[candidate_cols, target_cols])
print('spearman')
display(df.dropna().groupby('model_name').mean().corr('spearman').loc[candidate_cols, target_cols])

pearson


,real_interventions
traj_mae,-0.1894
whiteness,0.2737
cmd_whiteness,-0.1245


spearman


,real_interventions
traj_mae,0.2571
whiteness,0.1429
cmd_whiteness,-0.0286


#### Off-policy

In [68]:
pd.set_option('display.float_format', lambda x: '%.4f' % x)


target_cols = ['real_interventions']
candidate_cols = ['val_mae', 'elva_mae', 'elva_whiteness']
name_grouping = df.dropna().groupby('model_name').mean()

print('pearson')
pearson_corr = name_grouping.corr('pearson').loc[candidate_cols, target_cols]
pearson_pvals = name_grouping.corr(pearsonr_pval).loc[candidate_cols, target_cols]
pearson_corr.loc[:, 'p values'] = pearson_pvals[target_cols]
display(pearson_corr)

print('spearman')
spearman_corr = name_grouping.corr('spearman').loc[candidate_cols, target_cols]
spearman_pvals = name_grouping.corr(spearmanr_pval).loc[candidate_cols, target_cols]
spearman_corr.loc[:, 'p values'] = spearman_pvals[target_cols]
display(spearman_corr)


pearson


/home/nikita/miniconda3/envs/e2e/lib/python3.9/site-packages/scipy/stats/stats.py:4023: PearsonRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(PearsonRConstantInputWarning())


,real_interventions,p values
val_mae,0.6624,0.1517
elva_mae,-0.0719,0.8923
elva_whiteness,0.2759,0.5967


spearman


/home/nikita/miniconda3/envs/e2e/lib/python3.9/site-packages/scipy/stats/stats.py:4484: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


,real_interventions,p values
val_mae,0.7714,0.0724
elva_mae,-0.2571,0.6228
elva_whiteness,0.3143,0.5441


### Plots

In [26]:
import matplotlib.pyplot as plt
from metrics.metrics import calculate_lateral_errors

lat_errors = calculate_lateral_errors(model_ds.frames[:-1000], expert_ds.frames, only_autonomous=True)